In [1]:
import numpy as np
import progressbar 
import random 
import os
import cv2

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer

import keras
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.sequence import pad_sequences

import matplotlib.pyplot as plt
%matplotlib inline

from helpers import HDF5DatasetWriter
from helpers import Utils

Using TensorFlow backend.


In [25]:
output_path = "datasets/UCF-101-hdf5/fixed-sequence.hdf5"

In [3]:
root = 'datasets/UCF-101-frames-resize/'
include = ['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 
           'BalanceBeam', 'BandMarching', 'BaseballPitch', 'Basketball']

In [4]:
full_categories = [x[0] for x in os.walk(root) if x[0]][1:]
categories = [c for c in full_categories if c in [os.path.join(root, e) for e in include]]

In [5]:
categories

['datasets/UCF-101-frames-resize/ApplyEyeMakeup',
 'datasets/UCF-101-frames-resize/ApplyLipstick',
 'datasets/UCF-101-frames-resize/Archery',
 'datasets/UCF-101-frames-resize/BabyCrawling',
 'datasets/UCF-101-frames-resize/BalanceBeam',
 'datasets/UCF-101-frames-resize/BandMarching',
 'datasets/UCF-101-frames-resize/BaseballPitch',
 'datasets/UCF-101-frames-resize/Basketball']

In [6]:
def get_image(img_path):
    img = cv2.imread(img_path)
    return img

In [7]:
img = get_image("datasets/UCF-101-frames-resize/ApplyEyeMakeup/v_ApplyEyeMakeup_g06_c04_3.jpg")

In [9]:
img.shape

(60, 80, 3)

In [17]:
data = []
for c, category in enumerate(categories):
    images = [os.path.join(dp, f) for dp, dn, filenames 
              in os.walk(category) for f in filenames 
              if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
    
    current_squence_name = []
    next_squence_name = images[0].split('/')[2].split('_')[1:3]
    img_sequence = []
    img_count = 0
    img_max = 0
    img_min = 60
    
    for img_path in images:
        squence_name = img_path.split('/')[2].split('_')[1:3]
        current_squence_name = squence_name
        
        if current_squence_name == next_squence_name:
            img = get_image(img_path)
            img_count = img_count + 1
            if img_count <= 30:
                img_sequence.append(img)
        else:
            next_squence_name = current_squence_name
            data.append({'imgs':np.array(img_sequence), 'label':c})
            img_sequence = []
            if img_count < img_min:
                img_min = img_count
            if img_count > img_max:
                img_max = img_count
            img_count = 0
    data.append({'imgs':np.array(img_sequence), 'label':c})

In [18]:
print(img_min, img_max)

38 69


In [19]:
data = np.array(data)
data[4]['imgs'].shape

(30, 60, 80, 3)

In [21]:
x, y = np.array([t["imgs"] for t in data]), [t["label"] for t in data]

lb = LabelBinarizer()
y = lb.fit_transform(y)

In [27]:
def export_data(x, y, output_path, batch_size, buffer_size):
    dataset = HDF5DatasetWriter(x.shape, y.shape, output_path, bufSize = buffer_size)
    
    widgets = ["Exporting Data: ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
    pbar = progressbar.ProgressBar(maxval=x.shape[0], widgets=widgets).start()
    
    for i in np.arange(0, x.shape[0], batch_size):
        batchData = x[i:i + batch_size]
        batchLabels = y[i:i + batch_size]
        dataset.add(batchData, batchLabels)
        pbar.update(i)
    dataset.close()
    pbar.finish()

In [28]:
export_data(x, y, output_path, 32, 1000)

Exporting Data: 100% |##########################################| Time: 0:00:00
